This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [70]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [58]:
# First, import the relevant modules
import requests
import collections
import json
from datetime import datetime
import statistics

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [67]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
base_url = 'https://data.nasdaq.com/api/v3/datasets/'
API_target = 'FSE/AFX_X.json?start_date=2017-05-09&end_date=2017-05-09'
url = f'{base_url}{API_target}&api_key={API_KEY}'
r = requests.get(url)
json_data = r.json()

In [68]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json.dumps(json_data, indent = 3))

{
   "dataset": {
      "id": 10095370,
      "dataset_code": "AFX_X",
      "database_code": "FSE",
      "name": "Carl Zeiss Meditec (AFX_X)",
      "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
      "refreshed_at": "2020-12-01T14:48:09.907Z",
      "newest_available_date": "2020-12-01",
      "oldest_available_date": "2000-06-07",
      "column_names": [
         "Date",
         "Open",
         "High",
         "Low",
         "Close",
         "Change",
         "Traded Volume",
         "Turnover",
         "Last Price of the Day",
         "Daily Traded Units",
         "Daily Turnover"
      ],
      "frequency": "daily",
      "type": "Time Series",
      "premium": false,
      "limit": null,
      "transform": null,
      "column_index": null,
      "start_date": "2017-05-09",
      "end_date": "2017-05-09",
      "data": [
         [
            "2017-05-09",
   

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [71]:
API_target_2017 = 'FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31'
url_2017 = f'{base_url}{API_target_2017}&api_key={API_KEY}'
r_2017 = requests.get(url_2017)
json_data_2017 = r_2017.json()

data = json_data_2017['dataset']['data']

In [72]:
print(json_data_2017['dataset'].keys())

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


In [73]:
# Calculate highest opening price, lowest opening price, maximum price change
max_open_price = None
min_open_price = None
max_open_date = None
min_open_date = None
max_change = None
max_change_date = None

for day in data:
    date = day[0]
    open_price = day[1]
    high_price = day[2]
    low_price = day[3]
    if open_price is not None:
        if max_open_price is None or open_price > max_open_price:
            max_open_price = open_price
            max_open_date = date
        if min_open_price is None or open_price < min_open_price:
            min_open_price = open_price
            min_open_date = date
    if high_price is not None:
        if low_price is not None:
            day_change = high_price - low_price
            if max_change is None or day_change > max_change:
                max_change = day_change
                max_change_date = date

max_open_date_f = datetime.strptime(max_open_date, "%Y-%m-%d").strftime("%B %d, %Y")
min_open_date_f = datetime.strptime(min_open_date, "%Y-%m-%d").strftime("%B %d, %Y")
max_change_date_f = datetime.strptime(max_change_date, "%Y-%m-%d").strftime("%B %d, %Y")
print(f'The highest opening price for AFX_X in 2017 was \u20ac{max_open_price:.2f} on {max_open_date_f}.')
print(f'The lowest opening price for AFX_X in 2017 was \u20ac{min_open_price:.2f} on {min_open_date_f}.')
print(f'The maximum single-day price change for AFX_X in 2017 was \u20ac{max_change:.2f} on {max_change_date_f}.')

The highest opening price for AFX_X in 2017 was €53.11 on December 14, 2017.
The lowest opening price for AFX_X in 2017 was €34.00 on January 24, 2017.
The maximum single-day price change for AFX_X in 2017 was €2.81 on May 11, 2017.


In [74]:
# Calculate maximum day-to-day change in closing price
max_close_change = 0
today = None
yesterday = None

for i in range(1, len(data)):
    close_price_yesterday = data[i][4]
    close_price_today = data[i-1][4]
    close_change = abs(close_price_today - close_price_yesterday)
    if close_change > max_close_change:
        max_close_change = close_change
        today = data[i-1][0]
        yesterday = data[i][0]
        
today_f = datetime.strptime(today, "%Y-%m-%d").strftime("%B %d, %Y")         
yesterday_f = datetime.strptime(yesterday, "%Y-%m-%d").strftime("%B %d, %Y")
print(f'The largest change in AFX_X closing price from one day to the next was\n \u20ac{max_close_change:.2f} from {yesterday_f} to {today_f}.')

The largest change in AFX_X closing price from one day to the next was
 €2.56 from August 08, 2017 to August 09, 2017.


In [75]:
# Create a list of all the daily trading volumes
trading_volumes = [row[6] for row in data]

# Calculate the mean and median trading volume using the statistics module
mean_volume = statistics.mean(trading_volumes)
median_volume = statistics.median(trading_volumes)

print(f'The average daily trading volume of AFX_X in 2017 was {mean_volume:,.0f} shares \
and the \nmedian was {median_volume:,.0f} shares.')

The average daily trading volume of AFX_X in 2017 was 89,124 shares and the 
median was 76,286 shares.
